Recommendation systems are a collection of algorithms used to recommend items to users based on information taken from the user. These systems have become ubiquitous and can be commonly seen in online stores, movies databases and job finders. In this notebook, we will explore recommendation systems based on Collaborative Filtering and implement simple version of one using Python and the Pandas library.


<h1>Table of contents</h1>

<div class="alert alert-block alert-info" style="margin-top: 20px">
    <ol>
        <li><a href="https://#ref1">Acquiring the Data</a></li>
        <li><a href="https://#ref2">Preprocessing</a></li>
        <li><a href="https://#ref3">Collaborative Filtering</a></li>
    </ol>
</div>
<br>
<hr>


<a id="ref1"></a>

# Acquiring the Data


To acquire and extract the data, simply run the following Bash scripts:\
Dataset acquired from [GroupLens](http://grouplens.org/datasets/movielens/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkML0101ENSkillsNetwork20718538-2021-01-01). Let's download the dataset. To download the data, we will use **`!wget`** to download it from IBM Object Storage.\



In [1]:
#!wget -O moviedataset.zip https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML0101EN-SkillsNetwork/labs/Module%205/data/moviedataset.zip
#print('unziping ...')
#!unzip -o -j moviedataset.zip 

Now you're ready to start working with the data!


<hr>

<a id="ref2"></a>

# Preprocessing


First, let's get all of the imports out of the way:


In [2]:
#Dataframe manipulation library
import pandas as pd
#Math functions, we'll only need the sqrt function so let's import only that
from math import sqrt
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

Now let's read each file into their Dataframes:


In [3]:
#Storing the movie information into a pandas dataframe
movies_df = pd.read_csv('movies.csv')
#Storing the user information into a pandas dataframe
ratings_df = pd.read_csv('ratings.csv')

Let's also take a peek at how each of them are organized:


In [4]:
#Head is a function that gets the first N rows of a dataframe. N's default is 5.
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


So each movie has a unique ID, a title with its release year along with it (Which may contain unicode characters) and several different genres in the same field. Let's remove the year from the title column and place it into its own one by using the handy [extract](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.str.extract.html?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkML0101ENSkillsNetwork20718538-2021-01-01#pandas.Series.str.extract) function that Pandas has.


Let's remove the year from the **title** column by using pandas' replace function and store it in a new **year** column.


In [5]:
#Using regular expressions to find a year stored between parentheses
#We specify the parantheses so we don't conflict with movies that have years in their titles
movies_df['year'] = movies_df.title.str.extract('(\(\d\d\d\d\))',expand=False)
#Removing the parentheses
movies_df['year'] = movies_df.year.str.extract('(\d\d\d\d)',expand=False)
#Removing the years from the 'title' column
movies_df['title'] = movies_df.title.str.replace('(\(\d\d\d\d\))', '')
#Applying the strip function to get rid of any ending whitespace characters that may have appeared
movies_df['title'] = movies_df['title'].apply(lambda x: x.strip())

C:\Users\FRANCI~1\AppData\Local\Temp/ipykernel_29396/2727439841.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  movies_df['title'] = movies_df.title.str.replace('(\(\d\d\d\d\))', '')


Let's look at the result!


In [6]:
movies_df.head()

,movieId,title,genres,year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji,Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men,Comedy|Romance,1995
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II,Comedy,1995


With that, let's also drop the genres column since we won't need it for this particular recommendation system.


In [7]:
#Dropping the genres column
movies_df = movies_df.drop('genres',1)


C:\Users\FRANCI~1\AppData\Local\Temp/ipykernel_29396/3336070625.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  movies_df = movies_df.drop('genres',1)


In [8]:
#removing movies b4 2000 so it shows movies that are "more popular" 
print(movies_df['year'].size)
movies_df=movies_df.dropna()
print(movies_df['year'].size)
movies_df['year']= pd.to_numeric(movies_df['year'])
movies_df['year'].dtypes
movies_df= movies_df[movies_df['year'] >= 2000]


34208
34140


In [9]:
print(movies_df.size)

51462


Here's the final movies dataframe:


In [10]:
movies_df.head()

,movieId,title,year
2684,2769,"Yards, The",2000
3091,3177,Next Friday,2000
3104,3190,Supernova,2000
3139,3225,Down to You,2000
3142,3228,Wirey Spindell,2000


<br>


Next, let's look at the ratings dataframe.


In [11]:
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,169,2.5,1204927694
1,1,2471,3.0,1204927438
2,1,48516,5.0,1204927435
3,2,2571,3.5,1436165433
4,2,109487,4.0,1436165496


Every row in the ratings dataframe has a user id associated with at least one movie, a rating and a timestamp showing when they reviewed it. We won't be needing the timestamp column, so let's drop it to save on memory.


In [12]:
#Drop removes a specified row or column from a dataframe
ratings_df = ratings_df.drop('timestamp', 1)

C:\Users\FRANCI~1\AppData\Local\Temp/ipykernel_29396/1971122656.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  ratings_df = ratings_df.drop('timestamp', 1)


Here's how the final ratings Dataframe looks like:


In [13]:
ratings_df.head()

,userId,movieId,rating
0,1,169,2.5
1,1,2471,3.0
2,1,48516,5.0
3,2,2571,3.5
4,2,109487,4.0


<hr>

<a id="ref3"></a>

# Collaborative Filtering


Now it's time to start our work on recommendation systems.

The first technique we're going to take a look at is called **Collaborative Filtering**, which is also known as **User-User Filtering**. As hinted by its alternate name, this technique uses other users to recommend items to the input user. It attempts to find users that have similar preferences and opinions as the input and then recommends items that they have liked to the input. There are several methods of finding similar users (Even some making use of Machine Learning), and the one we will be using here is going to be based on the **Pearson Correlation Function**.

<img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML0101EN-SkillsNetwork/labs/Module%205/images/User_Item.png" width=800px>

The process for creating a User Based recommendation system is as follows:

*   Select a user with the movies the user has watched
*   Based on his rating of the movies, find the top X neighbours
*   Get the watched movie record of the user for each neighbour
*   Calculate a similarity score using some formula
*   Recommend the items with the highest score

Let's begin by creating an input user to recommend movies to:

Notice: To add more movies, simply increase the amount of elements in the userInput. Feel free to add more in! Just be sure to write it in with capital letters and if a movie starts with a "The", like "The Matrix" then write it in like this: 'Matrix, The' .


In [14]:
userInput = [
    
            {'title':'Toy Story', 'rating':4},
            {'title':'Alien', 'rating':4},
            {'title':'War of the Worlds', 'rating':3},
            {'title':'Spy Kids', 'rating':3},
            {'title':'Iron Man', 'rating':4},
            {'title':'Hangover, The', 'rating':3.5},
            {'title':'Star Wars: Episode VI - Return of the Jedi', 'rating':5},
            {'title':'Star Wars: Episode V - The Empire Strikes Back', 'rating':5},
            {'title':'Star Wars: Episode IV - A New Hope', 'rating':4},
            {'title':'Thor', 'rating':2}
    

   
         ] 
inputMovies = pd.DataFrame(userInput)
inputMovies

,title,rating
0,Toy Story,4.0
1,Alien,4.0
2,War of the Worlds,3.0
3,Spy Kids,3.0
4,Iron Man,4.0
5,"Hangover, The",3.5
6,Star Wars: Episode VI - Return of the Jedi,5.0
7,Star Wars: Episode V - The Empire Strikes Back,5.0
8,Star Wars: Episode IV - A New Hope,4.0
9,Thor,2.0


#### Add movieId to input user

With the input complete, let's extract the input movies's ID's from the movies dataframe and add them into it.

We can achieve this by first filtering out the rows that contain the input movies' title and then merging this subset with the input dataframe. We also drop unnecessary columns for the input to save memory space.


In [15]:
#Filtering out the movies by title
inputId = movies_df[movies_df['title'].isin(inputMovies['title'].tolist())]
inputId


,movieId,title,year
4139,4232,Spy Kids,2001
10231,34048,War of the Worlds,2005
12646,59315,Iron Man,2008
13288,64997,War of the Worlds,2005
13809,69122,"Hangover, The",2009
17074,86332,Thor,2011


In [16]:
#Then merging it so we can get the movieId. It's implicitly merging it by title.
inputMovies = pd.merge(inputId, inputMovies)
inputMovies

,movieId,title,year,rating
0,4232,Spy Kids,2001,3.0
1,34048,War of the Worlds,2005,3.0
2,64997,War of the Worlds,2005,3.0
3,59315,Iron Man,2008,4.0
4,69122,"Hangover, The",2009,3.5
5,86332,Thor,2011,2.0


In [17]:
#Dropping information we won't use from the input dataframe
inputMovies = inputMovies.drop('year', 1)

C:\Users\FRANCI~1\AppData\Local\Temp/ipykernel_29396/3541781103.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  inputMovies = inputMovies.drop('year', 1)


In [18]:
#Final input dataframe
#If a movie you added in above isn't here, then it might not be in the original 
#dataframe or it might spelled differently, please check capitalisation.
inputMovies

,movieId,title,rating
0,4232,Spy Kids,3.0
1,34048,War of the Worlds,3.0
2,64997,War of the Worlds,3.0
3,59315,Iron Man,4.0
4,69122,"Hangover, The",3.5
5,86332,Thor,2.0


#### The users who has seen the same movies

Now with the movie ID's in our input, we can now get the subset of users that have watched and reviewed the movies in our input.


In [19]:
#Filtering out users that have watched movies that the input has watched and storing it
userSubset = ratings_df[ratings_df['movieId'].isin(inputMovies['movieId'].tolist())]
userSubset.head()

,userId,movieId,rating
1175,15,59315,2.5
2328,19,59315,4.5
2866,23,59315,5.0
2884,23,86332,4.0
3860,37,34048,4.0


We now group up the rows by user ID.


In [20]:
#Groupby creates several sub dataframes where they all have the same value in the column specified as the parameter
userSubsetGroup = userSubset.groupby(['userId'])

Let's look at one of the users, e.g. the one with userID=1130.


In [21]:
userSubsetGroup.get_group(1130)

,userId,movieId,rating
105000,1130,59315,1.0
105028,1130,69122,2.5
105096,1130,86332,2.0


Let's also sort these groups so the users that share the most movies in common with the input have higher priority. This provides a richer recommendation since we won't go through every single user.


In [22]:
#Sorting it so users with movie most in common with the input will have priority
userSubsetGroup = sorted(userSubsetGroup,  key=lambda x: len(x[1]), reverse=True)

Now let's look at the first user.


In [23]:
userSubsetGroup[0:3]

[(6157,
          userId  movieId  rating
  568538    6157     4232     1.5
  568780    6157    34048     2.5
  568893    6157    59315     2.5
  568923    6157    64997     3.0
  568941    6157    69122     3.5
  569037    6157    86332     2.5),
 (6530,
          userId  movieId  rating
  612808    6530     4232     2.0
  613524    6530    34048     3.5
  613763    6530    59315     4.0
  613820    6530    64997     3.5
  613848    6530    69122     3.5
  613993    6530    86332     3.5),
 (20804,
           userId  movieId  rating
  1934407   20804     4232     3.0
  1934551   20804    34048     3.0
  1934652   20804    59315     4.0
  1934676   20804    64997     3.0
  1934699   20804    69122     4.5
  1934775   20804    86332     3.0)]

#### Similarity of users to input user

Next, we are going to compare all users (not really all !!!) to our specified user and find the one that is most similar.\
We're going to find out how similar each user is to the input through the **Pearson Correlation Coefficient**. It is used to measure the strength of a linear association between the two variables. The formula for finding this coefficient between sets X and Y with N values can be seen in the image below.

Why Pearson Correlation?

Pearson correlation is invariant to scaling, i.e. multiplying all elements by a nonzero constant or adding any constant to all elements. For example, if you have two vectors X and Y, then, pearson(X, Y) == pearson(X, 2 \* Y + 3). This is a pretty important property in recommendation systems because, for example, two users might rate two series of items totally differently in terms of absolute rates, but they would be similar users (i.e. with similar ideas) with similar rates in various scales .

![alt text](https://wikimedia.org/api/rest_v1/media/math/render/svg/bd1ccc2979b0fd1c1aec96e386f686ae874f9ec0 "Pearson Correlation")

The values given by the formula vary from r = -1 to r = 1, where 1 forms a direct correlation between the two entities (it means a perfect positive correlation) and -1 forms a perfect negative correlation.

In our case, a 1 means that the two users have similar tastes while a -1 means the opposite.


We will select a subset of users to iterate through. This limit is imposed because we don't want to waste too much time going through every single user.


In [24]:
userSubsetGroup = userSubsetGroup[0:10000]

Now, we calculate the Pearson Correlation between input user and subset group, and store it in a dictionary, where the key is the user Id and the value is the coefficient.


In [25]:
#Store the Pearson Correlation in a dictionary, where the key is the user Id and the value is the coefficient
pearsonCorrelationDict = {}

#For every user group in our subset
for name, group in userSubsetGroup:
    #Let's start by sorting the input and current user group so the values aren't mixed up later on
    group = group.sort_values(by='movieId')
    inputMovies = inputMovies.sort_values(by='movieId')
    #Get the N for the formula
    nRatings = len(group)
    #Get the review scores for the movies that they both have in common
    temp_df = inputMovies[inputMovies['movieId'].isin(group['movieId'].tolist())]
    #And then store them in a temporary buffer variable in a list format to facilitate future calculations
    tempRatingList = temp_df['rating'].tolist()
    #Let's also put the current user group reviews in a list format
    tempGroupList = group['rating'].tolist()
    #Now let's calculate the pearson correlation between two users, so called, x and y
    Sxx = sum([i**2 for i in tempRatingList]) - pow(sum(tempRatingList),2)/float(nRatings)
    Syy = sum([i**2 for i in tempGroupList]) - pow(sum(tempGroupList),2)/float(nRatings)
    Sxy = sum( i*j for i, j in zip(tempRatingList, tempGroupList)) - sum(tempRatingList)*sum(tempGroupList)/float(nRatings)
    
    #If the denominator is different than zero, then divide, else, 0 correlation.
    if Sxx > 0 and Syy > 0:
      
        pearsonCorrelationDict[name] = Sxy/sqrt(Sxx*Syy)
    else:
        pearsonCorrelationDict[name] = 0


In [26]:
pearsonCorrelationDict.items()

dict_items([(6157, 0.2075471698113216), (6530, 0.25697808437502445), (20804, 0.6981132075471691), (24874, 0.5046949386828395), (30771, -0.3646859721191216), (36946, 0.6376804093660599), (42438, 0.6301164651388237), (45232, -0.014560190658208001), (46750, 0.7985836518841521), (51287, 0.21520264493646457), (54212, 0.18355577455359023), (58040, 0.22509030421845364), (65908, 0.3820708742229554), (74985, 0.5611587602421327), (77738, 0.48463711111820856), (87907, 0.4758309514308862), (108213, 0.892752573112483), (112293, -0.111088444625541), (117248, -0.09712858623572153), (120519, 0.26667876121581063), (121309, 0.7329262651180467), (142730, 0.4941220250226691), (156725, 0.21251704297343013), (157690, 0.1615242910417859), (163118, 0.21279807063957248), (186680, -0.06662966046527602), (187463, 0.8795367562872938), (190114, 0.1535737792084868), (201670, 0.21365440436357994), (205892, 0.2928695358309708), (220231, 0), (224750, 0.5395415843971624), (227112, -0.05387724584403576), (232461, 0.0614

In [27]:
pearsonDF = pd.DataFrame.from_dict(pearsonCorrelationDict, orient='index')
pearsonDF.columns = ['similarityIndex']
pearsonDF['userId'] = pearsonDF.index
pearsonDF.index = range(len(pearsonDF))
pearsonDF.head()

,similarityIndex,userId
0,0.207547,6157
1,0.256978,6530
2,0.698113,20804
3,0.504695,24874
4,-0.364686,30771


#### The top x similar users to input user

Now let's get the top 50 users that are most similar to the input.


In [28]:
topUsers=pearsonDF.sort_values(by='similarityIndex', ascending=False)[0:50]
topUsers.head()

,similarityIndex,userId
1711,1.0,2340
3595,1.0,175706
2497,1.0,73184
3429,1.0,160338
2014,1.0,28591


Now, let's start recommending movies to the input user.

#### Rating of selected users to all movies

We're going to do this by taking the weighted average of the ratings of the movies using the Pearson Correlation as the weight. But to do this, we first need to get the movies watched by the users in our **pearsonDF** from the ratings dataframe and then store their correlation in a new column called \_similarityIndex". This is achieved below by merging of these two tables.


In [29]:
topUsersRating=topUsers.merge(ratings_df, left_on='userId', right_on='userId', how='inner')
topUsersRating.head()

,similarityIndex,userId,movieId,rating
0,1.0,2340,1,5.0
1,1.0,2340,5,4.0
2,1.0,2340,6,5.0
3,1.0,2340,7,5.0
4,1.0,2340,10,4.0


Now all we need to do is simply multiply the movie rating by its weight (the similarity index), then sum up the new ratings and divide it by the sum of the weights.

We can easily do this by simply multiplying two columns, then grouping up the dataframe by movieId and then dividing two columns:

It shows the idea of all similar users to candidate movies for the input user:


In [30]:
#Multiplies the similarity by the user's ratings
topUsersRating['weightedRating'] = topUsersRating['similarityIndex']*topUsersRating['rating']
topUsersRating.head()

,similarityIndex,userId,movieId,rating,weightedRating
0,1.0,2340,1,5.0,5.0
1,1.0,2340,5,4.0,4.0
2,1.0,2340,6,5.0,5.0
3,1.0,2340,7,5.0,5.0
4,1.0,2340,10,4.0,4.0


In [31]:
#Applies a sum to the topUsers after grouping it up by userId
tempTopUsersRating = topUsersRating.groupby('movieId').sum()[['similarityIndex','weightedRating']]
tempTopUsersRating.columns = ['sum_similarityIndex','sum_weightedRating']
tempTopUsersRating.head()

,sum_similarityIndex,sum_weightedRating
movieId,,
1,34.0,133.5
2,24.0,74.0
3,6.0,16.5
5,7.0,20.0
6,14.0,54.5


In [32]:
#Creates an empty dataframe
recommendation_df = pd.DataFrame()
#Now we take the weighted average
recommendation_df['weighted average recommendation score'] = tempTopUsersRating['sum_weightedRating']/tempTopUsersRating['sum_similarityIndex']
recommendation_df['movieId'] = tempTopUsersRating.index
recommendation_df.head()

,weighted average recommendation score,movieId
movieId,,
1,3.926471,1
2,3.083333,2
3,2.750000,3
5,2.857143,5
6,3.892857,6


Now let's sort it and see the top 20 movies that the algorithm recommended!


In [33]:
recommendation_df = recommendation_df.sort_values(by='weighted average recommendation score', ascending=False)
recommendation_df.head(10)

,weighted average recommendation score,movieId
movieId,,
148626,5.0,148626
106696,5.0,106696
93270,5.0,93270
1177,5.0,1177
417,5.0,417
5840,5.0,5840
95311,5.0,95311
95313,5.0,95313
95377,5.0,95377


In [34]:
movies_df.loc[movies_df['movieId'].isin(recommendation_df.head(50)['movieId'].tolist())]

,movieId,title,year
4240,4334,Yi Yi,2000
4278,4372,Crazy/Beautiful,2001
5162,5258,George Washington,2000
5171,5267,"Rookie, The",2002
9537,27857,As it is in Heaven (Så som i himmelen),2004
11317,48032,"Tiger and the Snow, The (La tigre e la neve)",2005
13308,65142,Loft,2008
13442,66203,He's Just Not That Into You,2009
13724,68519,S. Darko (S. Darko: A Donnie Darko Tale),2009
14477,72356,Partly Cloudy,2009


### Advantages and Disadvantages of Collaborative Filtering

##### Advantages

*   Takes other user's ratings into consideration
*   Doesn't need to study or extract information from the recommended item
*   Adapts to the user's interests which might change over time

##### Disadvantages

*   Approximation function can be slow
*   There might be a low amount of users to approximate
*   Privacy issues when trying to learn the user's preferences
